<a href="https://colab.research.google.com/github/dineshgulati933/portneuf_IWRRI/blob/main/ee_python/Pneuf_IrrMAPPER_huc12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
#  Python Script: Pneuf_IrrMAPPER_huc12.ipynb
#  Author        : Dinesh Gulati
#  Description   : Irrigated pixels for each HUC12.
# ============================================================

In [ ]:
# Extract irrigated area within each HUC12 for multiple years using IrrMapper dataset

In [ ]:
import ee

In [ ]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project = 'ee-dineshgulati333')

In [ ]:
wbd_huc8 = ee.FeatureCollection("USGS/WBD/2017/HUC08")
wbd_huc12 = ee.FeatureCollection("USGS/WBD/2017/HUC12")

aoi = wbd_huc8.filter(ee.Filter.inList('huc8', ['17040208']))

#Filter HUC12 features that are fully contained within the selected aoi
huc12_within_aoi = wbd_huc12.filter(ee.Filter.bounds(aoi))\
                                .filter(ee.Filter.contains(leftValue=aoi.geometry(),rightField='.geo'))

huc12_bbox = huc12_within_aoi.geometry().bounds()

In [ ]:
dataset = ee.ImageCollection('UMT/Climate/IrrMapper_RF/v1_2');
# irr = dataset.filterDate('2023-01-01', '2023-12-31')\
#                                 .filter(ee.Filter.bounds(huc4_bbox))

In [ ]:
def_proj = dataset.first().projection();

In [ ]:
years = range(2000, 2024)

tasks = []

for year in years:
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'

    dataset_yr = dataset\
        .filterDate(start_date, end_date)\
        .filterBounds(huc12_bbox)\
        .mosaic()\
        .unmask(1)\
        .remap([0,1],[1,0])\
        .setDefaultProjection(def_proj)\
        .set('Year', year)\
        #.set('Date', ee.Date.fromYMD(year,1,1).format('YYYY-MM-dd'))

    irr_stats = dataset_yr.reduceRegions(
        collection=huc12_within_aoi,
        reducer=ee.Reducer.frequencyHistogram(),
        scale=def_proj.nominalScale()
    ).map(lambda f: f.set('Year', dataset_yr.get('Year')))

    # Create export task
    task = ee.batch.Export.table.toDrive(
        collection=irr_stats,
        description=f'IRR_pixel_counts_HUC12_{year}',
        fileFormat='CSV',
        folder = 'Pneuf_IRR',
        selectors=['Year', 'huc12', 'histogram']
    )

    task.start()
    tasks.append(task)

    print(f'Export started for year {year}')


Export started for year 2000
Export started for year 2001
Export started for year 2002
Export started for year 2003
Export started for year 2004
Export started for year 2005
Export started for year 2006
Export started for year 2007
Export started for year 2008
Export started for year 2009
Export started for year 2010
Export started for year 2011
Export started for year 2012
Export started for year 2013
Export started for year 2014
Export started for year 2015
Export started for year 2016
Export started for year 2017
Export started for year 2018
Export started for year 2019
Export started for year 2020
Export started for year 2021
Export started for year 2022
Export started for year 2023
